In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math
import time

In [3]:
def exponential(matrix, c, gamma):
    matrix = matrix.astype(np.float32)
    matrix = c * np.power(matrix, gamma)
    matrix = np.clip(matrix, 0, 255).astype(np.uint8)
    return matrix

def ApplyMask(frame):
    frame_blur = cv.medianBlur(frame, 3)
    frame_hsv = cv.cvtColor(frame_blur, cv.COLOR_BGR2HSV)
    mask = cv.inRange(frame_hsv, (10,10,10), (40, 255, 255))
    frame_ = cv.bitwise_and(frame, frame, mask=mask)
    return frame_

In [4]:
frame = cv.Mat
frame_ = cv.Mat
paused = False
cap = cv.VideoCapture('../src/video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
times = [] 

h_min = 80
h_max = 90
s_min = 50
s_max = 255
v_min = 65
v_max = 255
# Define kernel size for morphological operations
kernel = np.ones((3, 3), np.uint8)

def nothing(x):
    pass

# Create a window and trackbars
cv.namedWindow('frame')
cv.createTrackbar('H_min', 'frame', h_min, 179, nothing)
cv.createTrackbar('H_max', 'frame', h_max, 179, nothing)
cv.createTrackbar('S_min', 'frame', s_min, 255, nothing)
cv.createTrackbar('S_max', 'frame', s_max, 255, nothing)
cv.createTrackbar('V_min', 'frame', v_min, 255, nothing)
cv.createTrackbar('V_max', 'frame', v_max, 255, nothing)

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
    start_time = time.time()
    # Get trackbar positions
    h_min = cv.getTrackbarPos('H_min', 'frame')
    h_max = cv.getTrackbarPos('H_max', 'frame')
    s_min = cv.getTrackbarPos('S_min', 'frame')
    s_max = cv.getTrackbarPos('S_max', 'frame')
    v_min = cv.getTrackbarPos('V_min', 'frame')
    v_max = cv.getTrackbarPos('V_max', 'frame')

    frame = cv.resize(frame, (640, 480))
    frame_blur = cv.medianBlur(frame, 3)
    # frame_exp = exponential(frame_blur, 0.3, 1.3)
    frame_hsv = cv.cvtColor(frame_blur, cv.COLOR_BGR2HSV)
    
    mask = cv.inRange(frame_hsv, (h_min, s_min, v_min), (h_max, s_max, v_max))
    # Apply erosion and dilation
    mask = cv.erode(mask, kernel, iterations=1)
    mask = cv.dilate(mask, kernel, iterations=1)
    frame_ = cv.bitwise_and(frame, frame, mask=mask)
    
    mask = cv.dilate(mask, kernel, iterations=1)    
   
    # Calcular os momentos do contorno
    m = cv.moments(mask)

    # Calcular as coordenadas do centroide
    cv_square_ctr_img = (int(m['m10'] / (m['m00'] + 1e-5)), int(m['m01'] / (m['m00'] + 1e-5)))
    #print(cv_square_ctr_img)       
    cv.circle(frame, cv_square_ctr_img, 50, (255, 0, 0), 2)
    end_time = time.time()

    processing_time = end_time - start_time
    times.append(processing_time)

    # Calculate the average processing time
    average_time = sum(times) / len(times)

    # Put the average processing time on the frame
    text = f"Avg time: {average_time:.4f} sec"
    cv.putText(frame, text, (10, frame.shape[0] - 20), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
    
    cv.imshow('frame', frame)
    cv.imshow('mask', mask)

    key = cv.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

cap.release()
cv.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kaio/.local/lib/python3.10/site-packages/cv2/qt/plugins"
